# Projeto 1 - Ciência dos Dados

Nome: Vitor Fortes Giuliano Riccetti 

Nome: Rodrigo Paoliello de Medeiros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'Chuteira_Puma.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Chuteira_Puma.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
# Faz a leitura do excel da parte do treinamento:

train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

train["Relevante"]= train["Relevante"].astype(str)
train["Irrelevante"]= train["Irrelevante"].astype(str)



In [4]:
# Faz a leitura do excel da parte do teste:

test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)



,Teste,classificacao
0,@gathlos de níver atrasado quero uma chuteira ...,Relevante
1,@neymarjrdepre @neymarjr @puma essa chuteira é...,Relevante
2,era só essa chuteira da puma na minha vida,Relevante
3,@manassescruz1 @juancrf19 @futebol_info @puma_...,Relevante
4,só a chuteira da puma na minha vida mesmo.,Relevante


In [5]:
# dividimos em duas categorias relevante e irrelevante:

Relevantes = ''.join(train.Relevante)

Irrelevantes = ' '.join(train.Irrelevante)



In [6]:
# Função para limpar a string


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    text = re.sub(r'@\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'kk\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;_“”]' # Note que os sinais [] são delimitadores de um conjunto.

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
shrink_whitespace_reg = re.compile(r'\s{2,}')

def clean_text(text_subbed):
    reg = re.compile(r'({})'.format(emoji_pat)) # line a
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', text_subbed)
    return shrink_whitespace_reg.sub(' ', result)




In [7]:
# Retira as pontuações e os links

Lista_R_clean = cleanup(Relevantes.lower())
Lista_R_clean = clean_text(Lista_R_clean)



In [8]:
# Retira as pontuações e os links

Lista_IR_clean = cleanup(Irrelevantes.lower())
Lista_IR_clean = clean_text(Lista_IR_clean)




In [9]:
# Serie de palavras Relevantes

Palavras_relevantes = pd.Series(Lista_R_clean.split())

len(Palavras_relevantes)

1759

In [10]:

Palavras_irrelevantes = pd.Series(Lista_IR_clean.split())

len(Palavras_irrelevantes)

2105

In [11]:
lista_total_palavras = []

for x in Palavras_relevantes:
    lista_total_palavras.append(x)
    
for y in Palavras_irrelevantes:
    lista_total_palavras.append(y)

total_palavras_n_repitir = pd.Series(lista_total_palavras)


In [12]:
total_lista_absoluta = total_palavras_n_repitir.value_counts(ascending=False)

len(total_lista_absoluta)

973

In [13]:
# Tabela absoluta dos Relevantes.

tabela_absoluta_relevantes = Palavras_relevantes.value_counts(ascending=False)


len(tabela_absoluta_relevantes)

561

In [14]:
# Tabela absoluta dos Irrelevantes.

tabela_absoluta_irrelevantes = Palavras_irrelevantes.value_counts(ascending=False)


tabela_absoluta_irrelevantes

chuteira    154
da          125
puma        113
a            83
neymar       56
           ... 
whats         1
seja          1
amigos        1
anos          1
deve          1
Length: 613, dtype: int64

In [15]:
# Frequencia relativa dos Relevantes 

tabela_relativa_relevantes = Palavras_relevantes.value_counts(True)



In [16]:
tabela_relativa_relevantes.sum()

1.0

In [17]:
# Frequencia relativa dos Irrelevantes

tabela_relativa_irrelevantes = Palavras_irrelevantes.value_counts(True)



In [18]:
tabela_relativa_irrelevantes.sum()

0.9999999999999999

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

RESPOSTA: 

- O produto que escolhemos é a nova chuteira do Neymar Jr chamada Puma King, mesmo modelo utilizadas pelos melhores jogadores do mundo como Pelé e Maradona com algumas modificações, devido as novas tecnologias de hoje.
- Os tweets foram classificados como relevantes se tivesse uma critica boa ou ruim a chuteira e os não relevantes são frase na qual nao a jugamentos a chuteira em si.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [19]:
# total de palavras
total = len(Palavras_relevantes) + len(Palavras_irrelevantes)

# Probabilidade de ser relevantes:
total_relevantes = len(Palavras_relevantes)/total

# Probabilidade de ser irrelevantes:
total_irrelevantes = len(Palavras_irrelevantes)/total

#total de palavras sem repitir
total_lista_absoluta

print(total_relevantes)
print(total_irrelevantes)

0.4552277432712215
0.5447722567287785


In [20]:
# Classificador de Tweets


classificador = []

for tweet in test.Teste:
    y = cleanup(tweet.lower())
    tweets = clean_text(y)
    r = 1
    i = 1
 
    for x in tweets:
        
        if x not in tabela_absoluta_relevantes:
            r = r * (0 + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        else:
            r = r *  (tabela_absoluta_relevantes[x] + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        if x not in tabela_absoluta_irrelevantes:
            i = i * (0 + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta))
        else:
            i = i * (tabela_absoluta_irrelevantes[x] + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta)) 
            
    R = total_relevantes * r
    I = total_irrelevantes * i
    if R > I: 
        classificador.append("Relevante")
    else:
        classificador.append("Irrelevante")




___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [21]:
test['classificador'] = classificador
test

,Teste,classificacao,classificador
0,@gathlos de níver atrasado quero uma chuteira ...,Relevante,Irrelevante
1,@neymarjrdepre @neymarjr @puma essa chuteira é...,Relevante,Relevante
2,era só essa chuteira da puma na minha vida,Relevante,Irrelevante
3,@manassescruz1 @juancrf19 @futebol_info @puma_...,Relevante,Relevante
4,só a chuteira da puma na minha vida mesmo.,Relevante,Irrelevante
...,...,...,...
294,no aguardo da proposta milionária na dm pra co...,Irrelevante,Irrelevante
295,"puma, chuteira preta hmm\nfedeu a hexa https:/...",Irrelevante,Relevante
296,serase o @neymarjr vai ressarcir meu dinheiro ...,Irrelevante,Relevante
297,só jogo lixo às 16hrs nessa rodada do br.\n\né...,Irrelevante,Irrelevante


In [23]:
pd.crosstab(test['classificacao'], test['classificador'], normalize='columns')

classificador,Irrelevante,Relevante
classificacao,,
Irrelevante,0.612022,0.405172
Relevante,0.387978,0.594828


In [1]:
print("Verdadeiro positivo: 59.48%")
print("Falso positivo: 38.79%")
print("Verdadeiros negativos: 61.20%")
print("Falsos negativos: 40.52%")
print('')
print("Acertos: ")
print("Erros: ")

Verdadeiro positivo: 59.48%
Falso positivo: 38.79%
Verdadeiros negativos: 61.20%
Falsos negativos: 40.52%

Acertos: 
Erros: 


___
### Concluindo




    Nós concluimos que o classificador criado pelo grupo está funcionando relativamente bem, pois como vimos em cima a maior parte está correta. As messagens com dupla negação e sarcasmo não foram interpretadas da forma que deveria ser, pois acreditamos que não era importante nessa situação inicial.
    O nosso plano de expansão será levar em consideração dupla negação e sarcasmo, para que o classificador fique cada vez mais completo. Além disso iremos aumentar o numero de grupos classificação, para conseguir ajudar mais o grupo de marketing da Puma, como por exemplo muito relevantes e muito irrelevantes. Outra melhoria que queremos fazer para melhorar nosso classificador é colocar todos os verbos no infinitivo pois assim consiguiremos aumentar a exatidão.
    Outro cenário que poderiamos usar um classificador desse tipo poderia ser para analisar a probabilidade de um candidato ganhar uma eleição.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/